In [ ]:
import random 
import chainer 
from chainer.datasets import mnist
from chainer import iterators 
import chainer.links as L
import chainer.functions as F
from chainer import training 
from chainer import optimizers 
from chainer.training import extensions
import numpy as np 

In [ ]:
class MLP(chainer.Chain):

    def __init__(self, n_mid_units=100, n_out=10):
        super(MLP, self).__init__()

        # パラメータを持つ層の登録
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def __call__(self, x):
        # データを受け取った際のforward計算を書く
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [ ]:
if chainer.cuda.available:
    chainer.cuda.cupy.random.seed(0)

train, test = mnist.get_mnist()

batchsize = 128 

train_iter = iterators.SerialIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)
device = 0 if chainer.cuda.available else -1 

net = MLP()
if device >= 0:
    net.to_gpu(device)

In [ ]:
classifier = L.Classifier(net)
optimizer = optimizers.SGD() 
optimizer.setup(classifier)
updater = training.StandardUpdater(train_iter, optimizer, device=device)
max_epoch = 10
trainer = training.Trainer(updater, (max_epoch,'epoch'), out='mnist_result')

In [ ]:
trainer.extend(extensions.LogReport())
trainer.extend(extensions.snapshot(filename='snapshot_epoch-{.updater.epoch}'))
trainer.extend(extensions.Evaluator(test_iter, classifier, device=device), name='val')
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'val/main/loss', 'val/main/accuracy', 'l1/W/data/std', 'elapsed_time']))
trainer.extend(extensions.ParameterStatistics(classifier.predictor.l1, {'std': np.std}))
trainer.extend(extensions.PlotReport(['l1/W/data/std'], x_key='epoch', file_name='std.png'))
trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.extend(extensions.dump_graph('main/loss'))

In [ ]:
trainer.run()

In [ ]:
from IPython.display import Image
Image(filename='mnist_result/loss.png')

In [ ]:
Image(filename='mnist_result/accuracy.png')